# Import Packgae

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import notebook


import talib as ta
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Line, Bar, Grid, Kline, Scatter

In [ ]:
# from pyecharts.globals import CurrentConfig, OnlineHostType
# CurrentConfig.ONLINE_HOST = OnlineHostType.NOTEBOOK_HOST

In [ ]:
sys.path.append("../../pysource/")
from Indicators import ZigZag
from Signals import (
    BTFSignal, VSignal, HSDSignal, WMSignal, TRPSignal, FABOSignal
)
from Strategies import FaboStrat
from Plot import (
    plot_candle, plot_line, plot_scatter, plot_bar
)

# Strategy test

## Single

In [ ]:
symbol = "I"
exchange = "XDCE"
trans_rate = 0.0001
gurat_rate = 0.1

In [ ]:
ts_raw = pd.read_csv(f"../../data/futures/{symbol}9999.{exchange}.csv")

In [ ]:
ts_raw = ts_raw.sort_values(by="date").reset_index(drop=True)
ts_raw.rename(columns={"date": "date_time"}, inplace=True)

In [ ]:
## 交易品种 + 窗口大小

In [ ]:
# Get ZZ list
syb_list = list(ts_raw.symbol.unique())
zigzag_list = list()
signal_list = list()
profit_list = dict()
for csyb in notebook.tqdm(syb_list):
    cts_raw = ts_raw[ts_raw["symbol"] == csyb]
    windows = 30
    zzo = ZigZag(windows)
    fabso = FABOSignal()
    fabsto = FaboStrat()
    for cur_price in cts_raw.to_dict(orient="records"):
        zzo.compute(cur_price)
        cur_zigzag = zzo.current_zigzag_
        if cur_zigzag is not None:
            fabso.recognize(cur_zigzag, cur_price)
            cur_signal = fabso.current_signal_
            if cur_signal is not None:
                fabsto.profit(cur_signal, cur_price)
                
    zigzag_list += zzo.zigzag_list_
    signal_list += fabso.signal_list_
    profit_list.update(fabsto._profit_dict)

In [ ]:
cf1 = pd.DataFrame(signal_list)

In [ ]:
cf1

In [ ]:
len(cf1)/ts_raw.date_time.apply(lambda x: x.split(" ")[0]).nunique()  # 平均每天交易次数

In [ ]:
pf1 = pd.DataFrame(profit_list).T.sort_index().reset_index().rename(columns={"index": "open_time"})

In [ ]:
pf1

In [ ]:
# 检查开仓点
(pf1.open_time != cf1["p3_time"].values).sum()

In [ ]:
tmp = pf1["profit"]
tmp1 = pf1["profit_rate"]

In [ ]:
# 单笔最大收益
tmp[tmp > 0].max()

In [ ]:
# 单笔最大亏损
tmp[tmp < 0].min()

In [ ]:
# 胜率
(tmp > 0).sum()/len(tmp)

In [ ]:
# 平均收益值
tmp.mean()

In [ ]:
# 平均收益率
tmp1.mean()

In [ ]:
# 收益值之和
tmp.sum()

In [ ]:
# 收益率之和
tmp1.sum()

In [ ]:
## 收益率计算

In [ ]:
# 带杠杆收益率计算方法1
m_rate = (tmp.mean())/(ts_raw["close"].mean())*(1/gurat_rate)  # 分母：价格*保证金比例倒数（杠杆倍数）  分子：单笔平均收益值
m_rate

In [ ]:
(tmp.mean())/(ts_raw["close"].mean())

In [ ]:
# 带杠杆收益率计算方法2
m_rate = (tmp1 * (1/gurat_rate)).mean()  # 收益率=涨跌比例*杠杆倍数
m_rate

In [ ]:
# 单笔交易成本率
cm_rate = (trans_rate*2)/gurat_rate  # 分母：保证金比例  分子：手续费比例*2 完整公式：（手续费率*交易金额（即：价格*交易乘数））/（保证金比率*交易金额（即：价格*交易乘数））
cm_rate

In [ ]:
(m_rate-cm_rate)*500000*0.004*3*220  # 单笔收益率 * 账户金额 * 持仓比例 * 每天平均交易次数 * 交易天数

## Total

In [ ]:
raw_dir = "../../data/futures/"
fabo_dir = "../../data/fabo_data/"

In [ ]:
_, _, filens = list(os.walk(raw_dir))[0]
profit_dict = dict()
for cf in notebook.tqdm(filens):
    syb = cf.split(".")[0]
    exh = cf.split(".")[1]
    # read
    crawd = pd.read_csv(f"{raw_dir}{cf}")
    crawd = crawd.sort_values(by="date").reset_index(drop=True)
    crawd.rename(columns={"date": "date_time"}, inplace=True)
    
    syb_list = list(crawd.symbol.unique())
    zigzag_list = list()
    highlow_list = list()
    signal_list = list()
    profit_list = dict()
    
    for csyb in notebook.tqdm(syb_list):
        ccrawd = crawd[crawd["symbol"] == csyb]
        # Get ZZ list
        windows = 30
        zzo = ZigZag(windows)
        fabso = FABOSignal()
        fabsto = FaboStrat()
        for cur_price in ccrawd.to_dict(orient="records"):
            zzo.compute(cur_price)
            cur_zigzag = zzo.current_zigzag_
            if cur_zigzag is not None:
                fabso.recognize(cur_zigzag, cur_price)
                cur_signal = fabso.current_signal_
                if cur_signal is not None:
                    fabsto.profit(cur_signal, cur_price)
                    
        zigzag_list += zzo.zigzag_list_
        highlow_list += zzo.highlow_list_
        signal_list += fabso.signal_list_
        profit_list.update(fabsto._profit_dict)
    
    # zigzag list
    czz_df = pd.DataFrame(zigzag_list)
    # high low list
    chl_df = pd.DataFrame(highlow_list)
    # signal of fabonacci
    cfabo_df = pd.DataFrame(signal_list)
    # profit of fabonacci
    cprofit = pd.DataFrame(profit_list).T.sort_index().reset_index().rename(columns={"index": "open_time"})
    
    czz_df.to_csv(f"{fabo_dir}{syb}_zigzag.csv", index=False)
    chl_df.to_csv(f"{fabo_dir}{syb}_highlow.csv", index=False)
    cfabo_df.to_csv(f"{fabo_dir}{syb}_signal.csv", index=False)
    cprofit.to_csv(f"{fabo_dir}{syb}_profit.csv", index=False)
    
    exh_days = crawd["date_time"].apply(lambda x: x.split(" ")[0]).nunique()
    
    if len(cprofit) > 0:
        profit_dict[syb] = dict()

        profit_dict[syb]["profit_sum"] = cprofit["profit"].sum()
        profit_dict[syb]["profit_mean"] = cprofit["profit"].mean()
        profit_dict[syb]["times_perday"] = len(cprofit)/exh_days
        profit_dict[syb]["profit_rate_sum"] = cprofit["profit_rate"].sum()
        profit_dict[syb]["profit_rate_mean"] = cprofit["profit_rate"].mean()
        profit_dict[syb]["times_win"] = (cprofit["profit"] > 0).sum()
        profit_dict[syb]["times_loss"] = (cprofit["profit"] <= 0).sum()
        profit_dict[syb]["profit_win_mean"] = (cprofit["profit"][cprofit["profit"] > 0]).sum()/profit_dict[syb]["times_win"]
        profit_dict[syb]["profit_loss_mean"] = (cprofit["profit"][cprofit["profit"] <= 0]).sum()/profit_dict[syb]["times_loss"]
        profit_dict[syb]["times_win_rate"] = profit_dict[syb]["times_win"]/len(cprofit)

In [ ]:
symbol_df = pd.read_csv("../../data/futures_dict.csv")
symbol_dict = dict(symbol_df.values)

In [ ]:
all_comdy = pd.DataFrame(profit_dict).T
all_comdy.index.names = ["futures"]
all_comdy["name"] = all_comdy.index.map(lambda x: symbol_dict[x[:-4]])

In [ ]:
cn_cols = ["总收益值", "平均收益值", "平均每天交易次数", "总收益率", "平均收益率", "盈利次数", "亏损次数", "盈利-平均金额", "亏损-平均金额", "胜率", "品种名称"]

In [ ]:
all_comdy.columns = cn_cols
all_comdy.index.names = ["品种代码"]

In [ ]:
all_comdy.to_excel(f"{fabo_dir}all_futures.xlsx")

# Plot

In [ ]:
cdl = plot_candle(ts_raw)

In [ ]:
volm = plot_bar(ts_raw)

In [ ]:
sigs = (
    Line()
    .add_xaxis(xaxis_data=ts_raw["date_time"].tolist())
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            is_scale=True
        ),
        toolbox_opts=opts.ToolboxOpts(),
        datazoom_opts=opts.DataZoomOpts(),
    )
)

for idx, crow in cf1.iterrows():
    v_tcols = ["p1_time", "p2_time", "p3_time"]
    v_pcols = ["p1_price", "p2_price", "p3_price"]
    csig = plot_line(crow[v_tcols].tolist(), crow[v_pcols].tolist(), crow["type"])
    sigs.overlap(csig)
    
for idx, crow in pf1.iterrows():
    v_tcols = ["open_time", "close_time"]
    v_pcols = ["open_price", "close_price"]
    csig = plot_line(crow[v_tcols].tolist(), crow[v_pcols].tolist(), crow["type"])
    sigs.overlap(csig)

In [ ]:
cdl.overlap(sigs)

In [ ]:
pv = (
    Grid(init_opts=opts.InitOpts(width="1400px", height="800px"))
    .add(
        cdl,
        grid_opts=opts.GridOpts(pos_left="5%", pos_right="1%", height="65%"),
    )
    .add(
        volm,
        grid_opts=opts.GridOpts(pos_left="5%", pos_right="1%", pos_top="75%", height="10%"),
    )
)

In [ ]:
pv.render_notebook()

In [ ]:
# 组合图
kl_base = (
    Kline()
    .add_xaxis(price_data.date_time.tolist())
    .add_yaxis(
        "",
        price_data[["open", "close", "low", "high"]].values.tolist(),
        itemstyle_opts=opts.ItemStyleOpts(
            color="#ef232a",
            color0="#14b143",
            border_color="#ef232a",
            border_color0="#14b143",
        ),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="line"),
        datazoom_opts=[
            opts.DataZoomOpts(
                is_show=False, type_="inside", xaxis_index=[0, 0], range_start=0, range_end=100, filter_mode="weakFilter"
            ),
            opts.DataZoomOpts(
                is_show=True, type_="slider", xaxis_index=[0, 1], range_start=0, range_end=100, pos_top="97%", filter_mode="weakFilter"
            ),
        ],
        title_opts=opts.TitleOpts(title="FGM"),
    )
)

for idx in signal_df.index:    
    sig_s = (
        Line()
        .add_xaxis(signal_df[["p1_time", "p2_time", "p3_time", "p4_time", "p5_time"]].loc[idx].values.tolist())
        .add_yaxis(
            series_name="", 
            y_axis=signal_df[["p1_price", "p2_price", "p3_price", "p4_price", "p5_price"]].loc[idx].values.tolist(),
            linestyle_opts = opts.LineStyleOpts(
                width=2,
                color="red" if signal_df["type"].loc[idx] == 1 else "green"
            )
        )
    )
    kls = kl_base.overlap(sig_s)

bar_vol = (
    Bar()
    .add_xaxis(price_data.date_time.tolist())
    .add_yaxis(
        series_name="",
        y_axis=[
            opts.BarItem(
                name="",
                value=x[2],
                itemstyle_opts={
                    "color": "#ef232a" if x[1] >= x[0] else "#14b143" 
                },
            )
            for x in price_data[["open", "close", "volume",]].values
        ],
        xaxis_index=1,
        yaxis_index=1,
        label_opts=opts.LabelOpts(is_show=False),
    )
)

pv = (
    Grid(init_opts=opts.InitOpts(width="1400px", height="800px"))
    .add(
        kls,
        grid_opts=opts.GridOpts(pos_left="5%", pos_right="1%", height="65%"),
    )
    .add(
        bar_vol,
        grid_opts=opts.GridOpts(pos_left="5%", pos_right="1%", pos_top="75%", height="10%"),
    )
)


In [ ]:
# # API for streamlit
# ste.st_echarts(json.loads(pv.dump_options_with_quotes()), height=600, width="100%")

# ALL Profit

In [ ]:
profit_dir = "../../data/profit_data/"
raw_dir = "../../data/futures/"

In [ ]:
_, _, filens = list(os.walk(profit_dir))[0]
profit_dict = dict()
for cf in notebook.tqdm(filens):
    syb = cf.split("_")[0]
    exh = (cf.split("_")[1]).split(".")[0]
    crawd = pd.read_csv(f"{raw_dir}{syb}9999.{exh}.csv")
    cprofit = pd.read_csv(f"{profit_dir}{cf}")
    
    exh_days = crawd["date"].apply(lambda x: x.split(" ")[0]).nunique()
    
    profit_dict[syb] = dict()
    
    profit_dict[syb]["profit_sum"] = cprofit["Profit"].sum()
    profit_dict[syb]["profit_mean"] = cprofit["Profit"].mean()
    profit_dict[syb]["times_perday"] = len(cprofit)/exh_days
    profit_dict[syb]["profit_rate_sum"] = cprofit["Profit_rate"].sum()
    profit_dict[syb]["profit_rate_mean"] = cprofit["Profit_rate"].mean()
    profit_dict[syb]["times_win"] = (cprofit["Profit"] > 0).sum()
    profit_dict[syb]["times_loss"] = (cprofit["Profit"] <= 0).sum()
    profit_dict[syb]["profit_win_mean"] = (cprofit["Profit"][cprofit["Profit"] > 0]).sum()/profit_dict[syb]["times_win"]
    profit_dict[syb]["profit_loss_mean"] = (cprofit["Profit"][cprofit["Profit"] <= 0]).sum()/profit_dict[syb]["times_loss"]
    profit_dict[syb]["times_win_rate"] = profit_dict[syb]["times_win"]/len(cprofit)

In [ ]:
symbol_df = pd.read_csv("../../data/futures_dict.csv")

In [ ]:
symbol_dict = dict(symbol_df.values)

In [ ]:
all_comdy = pd.DataFrame(profit_dict).T
all_comdy.index.names = ["futures"]

In [ ]:
all_comdy["name"] = all_comdy.index.map(lambda x: symbol_dict[x])

In [ ]:
cn_cols = ["总收益值", "平均收益值", "平均每天交易次数", "总收益率", "平均收益率", "盈利次数", "亏损次数", "盈利-平均金额", "亏损-平均金额", "胜率", "品种名称"]

In [ ]:
all_comdy.columns = cn_cols

In [ ]:
all_comdy.index.names = ["品种代码"]

In [ ]:
all_comdy.to_excel("../../data/all_futures.xlsx")